# Evaluate 
$ f(x, y) = 2x^2 - x^2y^2 + 3 $

$ g_0 = x ⋅ x $ 

$ g_1 = x ⋅ x $ 

$ g_2 = y ⋅ y $

$ g_3 = g_0 ⋅ 2 $

$ g_4 = g_1 ⋅ g_2 $

$ g_5 = g_2 - g_4 $

$ g_6 = g_4 + 3 $

---

$ c_0 = a_0 ⋅ b_0 $

$ c_1 = a_1 ⋅ b_1 $

$ c_2 = a_2 ⋅ b_2 $

$ c_3 = a_3 ⋅ 2 $

$ c_4 = a_4 ⋅ b_4 $

$ c_5 = a_5 - b_5 $

$ c_6 = a_6 + 3 $

---

$ x = a_0 = b_0 = a_1 = b_1 $

$ y = a_2 = b_2 $

$ g_0 = a_3 = c_0 $

$ g_1 = a_4 = c_1 $

$ g_2 = b_4 = c_2 $

$ g_3 = a_5 = c_3 $

$ g_4 = b_5 = c_4 $

$ g_5 = a_6 = c_5 $

$ g_6 = c_6 $

### Gate constrain equation

$ g_i = q_{Li}⋅a_i+q_{Ri}⋅b_i+q_{Mi}⋅(a_i⋅b_i)+q_{Ci}+q_{Oi}⋅c_i=0 $

$ g_0 = 0⋅a_0 + 0⋅b_0 + 1⋅(a_0⋅b_0) + 0 + (-1)⋅c_0 = 0 $

$ g_1 = 0⋅a_1 + 0⋅b_1 + 1⋅(a_1⋅b_1) + 0 + (-1)⋅c_1 = 0 $

$ g_2 = 0⋅a_2 + 0⋅b_2 + 1⋅(a_2⋅b_2) + 0 + (-1)⋅c_2 = 0 $

$ g_3 = 2⋅a_3 + 0⋅b_3 + 0⋅(a_3⋅b_3) + 0 + (-1)⋅c_3 = 0 $

$ g_4 = 0⋅a_4 + 0⋅b_4 + 1⋅(a_4⋅b_4) + 0 + (-1)⋅c_4 = 0 $

$ g_5 = 1⋅a_5 + (-1)⋅b_5 + 0⋅(a_5⋅b_5) + 0 + (-1)⋅c_5 = 0 $

$ g_6 = 1⋅a_6 + 0⋅b_6 + 0⋅(a_6⋅b_6) + 3 + (-1)⋅c_6 = 0 $

**selector vectors**

$ g_i = [q_{Li}, q_{Ri}, q_{Mi}, q_{Ci}, q_{Oi}] $

$ g_0 = [0, 0, 1, 0, -1] $

$ g_1 = [0, 0, 1, 0, -1] $

$ g_2 = [0, 0, 1, 0, -1] $

$ g_3 = [2, 0, 0, 0, -1] $

$ g_4 = [0, 0, 1, 0, -1] $

$ g_5 = [1, -1, 0, 0, -1] $

$ g_6 = [1, 0, 0, 3, -1] $

**let x = 2 y = 3**


| Vectors| a | b | c | qLi | qRi | qMi | qCi | qOi |
| --- | --- | --- | --- |--- | --- | --- | --- | --- |
| gate0 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate1 | 2 | 2 | 4 | 0 | 0 | 1 | 0 | -1 |
| gate2 | 3 | 3 | 9 | 0 | 0 | 1 | 0 | -1 |
| gate3 | 4 | 0 | 8 | 2 | 0 | 0 | 0 | -1 |
| gate4 | 4 | 9 | 36 | 0 | 0 | 1 | 0 | -1 |
| gate5 | 8 | 36 | -28 | 1 | -1 | 0 | 0 | -1 |
| gate6 | -28 | 3 | -25 | 1 | 0 | 0 | 3 | -1 |

In [43]:
import galois
import numpy as np

x = 2
y = 3

out = 2*x**2 - x**2*y**2 + 3
print(out)

a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]

ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, -1]

def to_galois_array(a, field):
    # normalize to positive values
    a = [x % field.order for x in a]
    return GF(a)

def to_vanishing_poly(roots, field):
    # Z^n - 1 = (Z - 1)(Z - w)(Z - w^2)...(Z - w^(n-1))
    return galois.Poly([1] + [0] * (len(roots)-1) + [-1], field=field)

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if type(v) == list else v
    return galois.lagrange_poly(x, y)

def pad_array(a, n):
    return a + [0]*(n - len(a))

n = len(a)
# we have 7 gates, but we need to pad to 2^k
n = 2**int(np.ceil(np.log2(n)))

# we have to pad the arrays to the same length
a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)

p = 97
GF = galois.GF(p)
w = GF.primitive_root_of_unity(n)

roots = GF([w**x for x in range(0, n)])

_A = to_poly(roots, a, GF)
_B = to_poly(roots, b, GF)
_C = to_poly(roots, c, GF)
ZH = to_vanishing_poly(roots, GF)

# a bit awkward, but we need to generate random values in GF for blinding polynomials
# b1, .., b9
b = galois.Poly.Random(degree=8, field=GF).coeffs

bA = galois.Poly(b[:2], field=GF)
bB = galois.Poly(b[2:4], field=GF)
bC = galois.Poly(b[4:6], field=GF)

A = _A + bA*ZH
B = _B + bB*ZH
C = _C + bC*ZH

QL = to_poly(roots, ql, GF)
QR = to_poly(roots, qr, GF)
QM = to_poly(roots, qm, GF)
QC = to_poly(roots, qc, GF)
QO = to_poly(roots, qo, GF)

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + qc(x) + c(x)*qo(x)
G = A*QL + B*QR + A*B*QM + QC + C*QO

T = G // ZH

print("--- Vanishing Polynomial ---")
print("ZH = ", ZH)

print("\n\n--- Blinding Polynomials ---")
print(f"bA = {bA}")
print(f"bB = {bB}")
print(f"bC = {bC}")

print("\n\n--- Wire Polynomials ---")
print(f"A = {_A}")
print(f"B = {_B}")
print(f"C = {_C}")

print("\n\n--- Blinded Wire Polynomials ---")
print(f"A' = {A}")
print(f"B' = {B}")
print(f"C' = {C}")

print("\n\n--- Selector Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")

print("\n\n--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(f"gate #{i} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}")

print("\n\n--- Target Polynomial ---")
print(f"T = {T}")

assert G % ZH == 0

-25
--- Vanishing Polynomial ---
ZH =  x^8 + 96


--- Blinding Polynomials ---
bA = 90x + 26
bB = 63x + 9
bC = 83x + 90


--- Wire Polynomials ---
A = 62x^7 + 81x^6 + 6x^5 + 8x^4 + 11x^3 + 48x^2 + 17x + 60
B = 6x^7 + 93x^6 + 17x^5 + 58x^4 + 65x^3 + 78x^2 + 54x + 19
C = 56x^7 + 16x^6 + 21x^5 + 5x^4 + 26x^3 + 95x^2 + 75x + 1


--- Blinded Wire Polynomials ---
A' = 90x^9 + 26x^8 + 62x^7 + 81x^6 + 6x^5 + 8x^4 + 11x^3 + 48x^2 + 24x + 34
B' = 63x^9 + 9x^8 + 6x^7 + 93x^6 + 17x^5 + 58x^4 + 65x^3 + 78x^2 + 88x + 10
C' = 83x^9 + 90x^8 + 56x^7 + 16x^6 + 21x^5 + 5x^4 + 26x^3 + 95x^2 + 89x + 8


--- Selector Polynomials ---
QL = 26x^7 + 82x^6 + 61x^5 + 24x^4 + 17x^3 + 39x^2 + 90x + 49
QR = 8x^7 + 70x^6 + 79x^5 + 85x^4 + 89x^3 + 27x^2 + 18x + 12
QM = 17x^7 + 85x^6 + 60x^5 + 85x^4 + 37x^3 + 85x^2 + 80x + 37
QC = 16x^7 + 36x^6 + 81x^5 + 61x^4 + 16x^3 + 36x^2 + 81x + 61
QO = 18x^7 + 70x^6 + 89x^5 + 12x^4 + 79x^3 + 27x^2 + 8x + 84


--- Gate Constraints Polynomial ---
G = 69x^25 + 57x^24 + 56x^23 + 19x^

### Permutation constraint

In [44]:
def print_sigma(sigma, a, b, c, r):
    group_size = len(sigma) // 3
    padding = 6

    print(f"{' w'} | {'value':{padding}} | {'i':{padding}} | {'sigma(i)':{padding}}")

    for i in range(0, group_size):
        print(f"a{i} | {a[i]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(group_size, 2 * group_size):
        print(f"b{i - group_size} | {b[i - group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(2 * group_size, 3 * group_size):
        print(f"c{i - 2 * group_size} | {c[i - 2 * group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[0], ai[1]: ai[1], ai[2]: ai[2], ai[3]: ci[0], ai[4]: ci[1], ai[5]: ci[3], ai[6]: ci[5], ai[7]: ai[7],
    bi[0]: bi[0], bi[1]: bi[1], bi[2]: bi[2], bi[3]: bi[3], bi[4]: ci[2], bi[5]: ci[4], bi[6]: bi[6], bi[7]: bi[7],
    ci[0]: ai[3], ci[1]: ai[4], ci[2]: bi[4], ci[3]: ai[5], ci[4]: bi[5], ci[5]: ai[6], ci[6]: ci[6], ci[7]: ci[7],
}

c0_roots = roots
c1_roots = roots * 2
c2_roots = roots * 4

r = np.concatenate((c0_roots, c1_roots, c2_roots))

check = set()
for w in r:
    if int(w) in check:
        print(f"Duplicate root {w} in {r}")
        assert False
    check.add(int(w))

sigma0 = GF([r[sigma[i]] for i in range(0, n)])
sigma1 = GF([r[sigma[i + n]] for i in range(0, n)])
sigma2 = GF([r[sigma[i + 2 * n]] for i in range(0, n)])

print_sigma(sigma, a, b, c, r)

print("\n\n--- Cosest ---")
print(f"c0 = {c0_roots}")
print(f"c1 = {c1_roots}")
print(f"c2 = {c2_roots}")

print("\n\n--- Sigma ---")
print(f"sigma0 = {sigma0}")
print(f"sigma1 = {sigma1}")
print(f"sigma2 = {sigma2}")

S0 = to_poly(roots, sigma0, GF)
S1 = to_poly(roots, sigma1, GF)
S2 = to_poly(roots, sigma2, GF)

I0 = to_poly(roots, c0_roots, GF)
I1 = to_poly(roots, c1_roots, GF)
I2 = to_poly(roots, c2_roots, GF)

beta = GF(15)
gamma = GF(32)

padding = 2
for i in range(0, len(roots)):
    s = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   A({roots[i]:{padding}}) = {A(roots[i]):{padding}} "
    s += f"   B({roots[i]:{padding}}) = {B(roots[i]):{padding}} "
    s += f"   C({roots[i]:{padding}}) = {C(roots[i]):{padding}} "
    s += f"   I0({roots[i]:{padding}}) = {I0(roots[i]):{padding}} "
    s += f"   I1({roots[i]:{padding}}) = {I1(roots[i]):{padding}} "
    s += f"   I2({roots[i]:{padding}}) = {I2(roots[i]):{padding}} "
    s += f"   S0({roots[i]:{padding}}) = {S0(roots[i]):{padding}} "
    s += f"   S1({roots[i]:{padding}}) = {S1(roots[i]):{padding}} "
    s += f"   S2({roots[i]:{padding}}) = {S2(roots[i]):{padding}} "
    print(s)


f = (A + I0 * beta + gamma) * (B + I1 * beta + gamma) * (C + I2 * beta + gamma)
g = (A + S0 * beta + gamma) * (B + S1 * beta + gamma) * (C + S2 * beta + gamma)

print("\n\n--- f(x) ---")
print(f"f(x) = {f}")

print("\n\n--- g(x) ---")
print(f"g(x) = {g}")

acc_poly_eval = [GF(1)]
for i in range(0, n):
    acc_poly_eval.append(
        acc_poly_eval[-1] * f(roots[i]) / g(roots[i])
    )

assert acc_poly_eval.pop() == GF(1)
ACC = galois.lagrange_poly(roots, GF(acc_poly_eval))

print("\n\n--- Accumulator Polynomial ---")
print(f"ACC(x) = {ACC}")

# blinding polynomial for accumulator
bZ = galois.Poly(b[6:], field=GF)
Z = bZ * ZH + ACC

print("\n\n--- Z ---")
print(f"Z(x) = {Z}")

for r in roots:
    print(f"Z({r}) = {Z(r)}")


 w | value  | i      | sigma(i)
a0 |      2 |      1 |      1
a1 |      2 |     64 |     64
a2 |      3 |     22 |     22
a3 |      4 |     50 |      4
a4 |      4 |     96 |     62
a5 |      8 |     33 |      6
a6 |    -28 |     75 |     35
a7 |      0 |     47 |     47
-- | --     | --     | --    
b0 |     90 |      2 |      2
b1 |     26 |     31 |     31
b2 |     63 |     44 |     44
b3 |      9 |      3 |      3
b4 |     83 |     95 |     88
b5 |     90 |     66 |     93
b6 |     40 |     53 |     53
b7 |     85 |     94 |     94
-- | --     | --     | --    
c0 |      4 |      4 |     50
c1 |      4 |     62 |     96
c2 |      9 |     88 |     95
c3 |      8 |      6 |     33
c4 |     36 |     93 |     66
c5 |    -28 |     35 |     75
c6 |    -25 |      9 |      9
c7 |      0 |     91 |     91


--- Cosest ---
c0 = [ 1 64 22 50 96 33 75 47]
c1 = [ 2 31 44  3 95 66 53 94]
c2 = [ 4 62 88  6 93 35  9 91]


--- Sigma ---
sigma0 = [ 1 64 22  4 62  6 35 47]
sigma1 = [ 2 31 44  3 88 93

In [45]:
from py_ecc.optimized_bls12_381 import G1, multiply, add

# powers of tau
tau = 45

powers_of_tau = [multiply(G1, tau**i) for i in range(0, G.degree)]

def msm(powers, poly: galois.Poly):
    coeff = poly.coeffs[::-1]
    res = multiply(G1, 0)
    for i in range(0, len(coeff)):
        res = add(res, multiply(powers[i], int(coeff[i])))

    return res

A_commitment = msm(powers_of_tau, A)
B_commitment = msm(powers_of_tau, B)
C_commitment = msm(powers_of_tau, C)
T_commitment = msm(powers_of_tau, T)

print("\n\n--- Commitments ---")
print(f"A_commitment = {A_commitment}")
print(f"B_commitment = {B_commitment}")
print(f"C_commitment = {C_commitment}")
print(f"T_commitment = {T_commitment}")



--- Commitments ---
A_commitment = (2045294285358329714359378283588320634874323328908949882758870726662088243338611221492174358313910440136151214755701, 1770652684565202969545029418765254677318170940126871394516373743616671635973405326782691651577054534131077161335155, 3018546892142785587106499405087345508551824384067752527182909389033002265451913828455626147537241056629818021061539)
B_commitment = (831608378347425493000038755224243780422175469755459617657858619470846569159249298126232363255968463954348639705274, 3263476804638795859944148169230033538838669854410495238573528528862519802779533989182977246166850769862057083387089, 3863494083203046847849495204895166355206788305567404835504561644802295978065460832320510904908160488283226629409921)
C_commitment = (1298173998817604132970137733901445804428720484689623111154724081689655133069832539607505968101772134791725447306186, 1307795479073000691822951091514389853121151876186949262321760439686719677550373424308704345451753722855474752501